# Kurt Schwarz Foto-Bot @schwarzPhotoBot@mastodon.social

In [ ]:
import requests
import re

In [ ]:
from datetime import datetime, timedelta

def generate_dates(days):
    '''Funktion die eine Liste von <days> aufeinanderfolgenden Datumsangaben
    erzeugt.'''
    today = datetime.now() + timedelta(days = 1)
    return [(today + timedelta(days=i)).strftime("%d.%m.%Y") for i in range(days)]

In [ ]:
# über die Suchmaske der Weboberfläche konfigurierte Anfrage
# JSON erreichbar mit Strg + Rechtsklick => Exportieren
# Abgerufen wurden Objekte des Industriesalons Schöneweide mit
# geeigneter Lizenz (CC BY-SA)
def getMDdata(offset, rows):
    apiCall = f"https://berlin.museum-digital.de/json/objects?&s=collection%3A1120&startwert={offset}&gbreitenat={rows}"
    return requests.get(apiCall).json()

def objectMetadata(objectID):
    url = f"https://berlin.museum-digital.de/json/object/{objectID}"
    res = requests.get(url)
    return res.json()

def createPostContent(objectID):
    objectDict = objectMetadata(objectID)
    objName = objectDict.get('object_name')
    objName = re.sub('\.(\s+)?SW-Foto.*','',objName)
    objName = re.sub('[\r\n]+',' ',objName)
    objDesc = objectDict.get('object_description')
    objDesc = re.sub('\.(\s+)?SW-Foto.*','',objDesc)
    objDesc = re.sub('[\r\n]+',' ',objDesc)
    
    mainImg = [ _ for _ in objectDict['object_images'] if _['is_main'] == "j"][0]
    mainImgPreview = "https://asset.museum-digital.org/berlin/" + mainImg['folder'] + "/" + mainImg['filename_loc']
    rights = re.sub('[\r\n]+',' ', mainImg['rights'])
    
    owner = mainImg['owner']
    creator = mainImg['creator']
    
    if creator == "":
        creator = "Kurt Schwarz"
    
    copyrightLine = f"© {creator}, {rights}"
    
    hashtags = "#berlin #photography #industriesalon"
    
    textContent = f"{objName.rstrip('.')}. {objDesc} ({copyrightLine}). Erfahren Sie mehr über das Objekt auf #museumdigital unter https://berlin.museum-digital.de/object/{objectID} {hashtags}"
    
    if len(textContent) > 480:
        textContent = f"{objName} ({copyrightLine}). Erfahren Sie mehr über das Objekt auf #museumdigital unter https://berlin.museum-digital.de/object/{objectID} {hashtags}"
    
    return textContent, mainImgPreview

In [ ]:
createPostContent("101490")

In [ ]:
total = getMDdata(0,1)[0].get('total')

In [ ]:
rows = 100
autodone_table = list()
dates = generate_dates(total) 

autodone_table.append(["Date", "Time", "Text", "Picture", "Pic description"])

for i in range(0,total + 1, rows):
    print(i)
    res = getMDdata(i, rows)
    for r in res:
        objID = r['objekt_id']
        print(objID)
        postContent = createPostContent(objID)
        
        date = dates.pop(0)
        time = "09:00"
        text = postContent[0]
        picture = postContent[1]
        pictureDesc = ""
        
        row = [date, time, text, picture, pictureDesc]
        autodone_table.append(row)

In [ ]:
# shuffle post order
from random import shuffle
content = [_[1:] for _ in autodone_table[1:]]
shuffle(content)
shuffled_autodone_table = [[_[0]] + c for (_,c) in zip(autodone_table[1:],content)]
shuffled_autodone_table.insert(0, ["Date", "Time", "Text", "Picture", "Pic description"])

In [ ]:
import csv
try:
    ## Bei lokalem Betrieb kann Tabelle als csv gespeichert werden
    OUT = open("plewka_2025_02_17.tsv", "w")
    writer = csv.writer(OUT, delimiter='\t')
    writer.writerows(shuffled_autodone_table)
    OUT.close()
    ## Wird der Code in der JupyterLite-Umgebung ausgeführt, kann
    ## lokal nichts gespeichert werden. Daher wird die Liste einfach
    ## nur unter der Zelle ausgegeben.
except OSError:
    for row in shuffled_autodone_table:
        print("\t".join(row))

In [ ]:
# exclude aready posted objects
exclude = ["96966","101490"]

In [ ]:
import ollama

In [ ]:
ollama.list()

In [ ]:
for _ in shuffled_autodone_table[1:5]:
    string = f"{_[2]}"
    resp = ollama.chat(model='llava:34b', messages=[{'role': 'user', 'content': f'''Based on the following
    string, provide a maximum of five hashtags that best describe the object. 
    Output only the list of hashtags, nothing else. String: {string}'''}])
    print(string, resp['message']['content'])

In [ ]:
ollama.chat(model='llama3.2', messages=[{'role': 'user', 'content': 'Provide '}])

In [ ]:
for _ in shuffled_autodone_table:
    print(_)